In [3]:
PATH_QASM_A = "artifacts/test_a.qasm"
PATH_QASM_B = None  # "artifacts/test_b.qasm"

PATH_PYTHON_FILE = "artifacts/file.py"

In [ ]:
# Parameters
PATH_QASM_A = "reports/v001_manual/2024_11_04__14_48/qiskit_circuit_5q_10g_9_160f04_cfe9f1_error_min_qc_qiskit.qasm"
PATH_QASM_B = "reports/v001_manual/2024_11_04__14_48/qiskit_circuit_5q_10g_9_160f04_cfe9f1_error_min_qc_pytket.qasm"
PATH_PYTHON_FILE = "program_bank/v007/2024_10_31__17_51__qiskit/qiskit_circuit_5q_10g_9_160f04_cfe9f1_error_min.py"

In [8]:

# consider all the imports relative to the current folder
from pathlib import Path
PATH_QASM_A = str(Path(PATH_QASM_A).name)
PATH_QASM_B = str(Path(PATH_QASM_B).name)
PATH_PYTHON_FILE = str(Path(PATH_PYTHON_FILE).name)

## File Content

In [9]:
from termcolor import colored


def extract_between_tags(content: str, start_tag: str, end_tag: str) -> str:
    start_index = content.find(start_tag) + len(start_tag)
    end_index = content.find(end_tag)
    return content[start_index:end_index]


def print_and_return_content(
        file_name, color: str, start_tag: str = None, end_tag: str = None) -> str:
    print(colored("Filename: ", color), file_name)
    with open(file_name, "r") as f:
        content = f.read()
        if start_tag and end_tag:
            content = extract_between_tags(content, start_tag, end_tag)
        print(colored(content, color))
    print("\n")
    return content


qasm_a_str = print_and_return_content(PATH_QASM_A, color="green")
if PATH_QASM_B:
    qasm_b_str = print_and_return_content(PATH_QASM_B, color="blue")

py_file_str = print_and_return_content(
    PATH_PYTHON_FILE, color="magenta", start_tag="# <START_GATES>",
    end_tag="import os")

Filename:  qiskit_circuit_5q_10g_9_160f04_cfe9f1_error_min_qc_qiskit.qasm
OPENQASM 2.0;
include "qelib1.inc";
qreg cin[1];
qreg a[2];
qreg b[2];
cx a[0],b[0];



Filename:  qiskit_circuit_5q_10g_9_160f04_cfe9f1_error_min_qc_pytket.qasm
OPENQASM 2.0;
include "qelib1.inc";

qreg a[2];
qreg b[2];
qreg cin[1];
cx a[0],b[0];



Filename:  qiskit_circuit_5q_10g_9_160f04_cfe9f1_error_min.py


qc = qasm2.loads(
"""
include "qelib1.inc";
qreg cin[1];
qreg a[2];
qreg b[2];
cx a[0],b[0];
"""
    , custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)






## Circuit Diagram

In [15]:
from qiskit.qasm2 import load
from qiskit import qasm2

circuit_a = load(
    PATH_QASM_A, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
print(circuit_a)
if PATH_QASM_B:
    circuit_b = load(
        PATH_QASM_B, custom_instructions=qasm2.LEGACY_CUSTOM_INSTRUCTIONS)
    print(circuit_b)

          
cin: ─────
          
a_0: ──■──
       │  
a_1: ──┼──
     ┌─┴─┐
b_0: ┤ X ├
     └───┘
b_1: ─────
          
          
a_0: ──■──
       │  
a_1: ──┼──
     ┌─┴─┐
b_0: ┤ X ├
     └───┘
b_1: ─────
          
cin: ─────
          


## Compare Semantics

### QCEC Oracle

In [11]:
from mqt import qcec
if PATH_QASM_B:
    res = qcec.verify(
        PATH_QASM_A, PATH_QASM_B,
        transform_dynamic_circuit=True)
    print(colored(f"QCEC result: {res.equivalence}", "magenta"))
    print(res)

QCEC result: not_equivalent
{
  "check_time": 0.01877822,
  "checkers": [
    {
      "checker": "decision_diagram_alternating",
      "equivalence": "no_information",
      "max_nodes": 0,
      "runtime": 0.0
    },
    {
      "checker": "zx",
      "equivalence": "probably_not_equivalent",
      "runtime": 7.1892e-05
    },
    {
      "checker": "decision_diagram_simulation",
      "equivalence": "no_information",
      "max_nodes": 0,
      "runtime": 0.0
    },
    {
      "checker": "decision_diagram_simulation",
      "equivalence": "no_information",
      "max_nodes": 0,
      "runtime": 0.0
    },
    {
      "checker": "decision_diagram_simulation",
      "equivalence": "not_equivalent",
      "max_nodes": 6,
      "runtime": 1.9539e-05
    },
    {
      "checker": "decision_diagram_simulation",
      "equivalence": "no_information",
      "max_nodes": 0,
      "runtime": 0.0
    },
    {
      "checker": "decision_diagram_simulation",
      "equivalence": "no_information"

### Simulation (Qiskit)

In [12]:
from qiskit.quantum_info import Statevector

sv_a = Statevector.from_instruction(circuit_a)
print(colored(PATH_QASM_A, "green"))
print(colored("Statevector A: ", "green"), sv_a)

if PATH_QASM_B:
    sv_b = Statevector.from_instruction(circuit_b)
    print(colored(PATH_QASM_B, "blue"))
    print(colored("Statevector B: ", "blue"), sv_b)

# compare the statevectors
probabilities_a = sv_a.probabilities_dict()
print(colored(PATH_QASM_A, "green"))
print(colored("Probabilities A: ", "green"), probabilities_a)
if PATH_QASM_B:
    probabilities_b = sv_b.probabilities_dict()
    print(colored(PATH_QASM_B, "blue"))
    print(colored("Probabilities B: ", "blue"), probabilities_b)

qiskit_circuit_5q_10g_9_160f04_cfe9f1_error_min_qc_qiskit.qasm
Statevector A:  Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2, 2, 2, 2))
qiskit_circuit_5q_10g_9_160f04_cfe9f1_error_min_qc_pytket.qasm
Statevector B:  Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
             0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2, 2, 2, 2))
qiskit_circuit_5q_10g_9_160f04_cfe9f1_error_min_qc_qiskit.qasm
Probabilities A:  {'00000': 1.0}
qiskit_circuit_5q_10g_9_160f04_cfe

In [ ]:
from pytket.qasm import circuit_to_qasm_str
from pytket.qasm import circuit_from_qasm_str
qasm_str = """
include "qelib1.inc";
qreg cin[1];
qreg a[2];
qreg b[2];
cx a[0],b[0];
"""

circuit = circuit_from_qasm_str(qasm_str)
qasm_str_reordered_by_pytket = circuit_to_qasm_str(circuit)
print(qasm_str_reordered_by_pytket)

OPENQASM 2.0;
include "qelib1.inc";

qreg a[2];
qreg b[2];
qreg cin[1];
cx a[0],b[0];



In [ ]:
# add one X gate at the register in position 1
from qiskit import QuantumCircuit
from qiskit.qasm2 import loads
qc_input = QuantumCircuit(5)
qc_input.x(1)


circuit_a = loads(qasm_str)

circuit_b = loads(qasm_str_reordered_by_pytket)


# original circuit
qc_w_input_original = circuit_a.compose(
    qc_input, front=True)
print(qc_w_input_original)

# imported circuit
qc_w_input_buggy = circuit_b.compose(
    qc_input, front=True)
print(qc_w_input_buggy)

# compare the statevectors
sv_w_input_original = Statevector.from_instruction(qc_w_input_original)
probabilities_w_input_original = sv_w_input_original.probabilities_dict()
print("Probabilities W Input Original: ", probabilities_w_input_original)

sv_w_input_buggy = Statevector.from_instruction(qc_w_input_buggy)
probabilities_w_input_buggy = sv_w_input_buggy.probabilities_dict()
print(
    "Probabilities W Input - After PyTket Reordering: ",
    probabilities_w_input_buggy)

               
cin: ──────────
     ┌───┐     
a_0: ┤ X ├──■──
     └───┘  │  
a_1: ───────┼──
          ┌─┴─┐
b_0: ─────┤ X ├
          └───┘
b_1: ──────────
               
               
a_0: ───────■──
     ┌───┐  │  
a_1: ┤ X ├──┼──
     └───┘┌─┴─┐
b_0: ─────┤ X ├
          └───┘
b_1: ──────────
               
cin: ──────────
               
Probabilities W Input Original:  {'01010': 1.0}
Probabilities W Input - After PyTket Reordering:  {'00010': 1.0}
